<a href="https://colab.research.google.com/github/Leon-web-net/Learning_ML/blob/main/Ensemble_Methods/Gradient_Boosting_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [10]:
# import kagglehub
# import os
# import shutil

# # Download latest version
# path = kagglehub.dataset_download("nimapourmoradi/raisin-binary-classification")
# print("Dataset downloaded to:", path)

# # Define source and destination
# src = os.path.join(path, "Raisin_Dataset.csv")   # adjust name if different
# dst = "./drive/MyDrive/datasets/raisan.csv"

# # Make sure target folder exists
# os.makedirs(os.path.dirname(dst), exist_ok=True)

# # Copy file
# shutil.copy(src, dst)

# print(f"File saved to {dst}")

100%|██████████| 110k/110k [00:00<00:00, 40.0MB/s]

Extracting files...
Dataset downloaded to: /root/.cache/kagglehub/datasets/nimapourmoradi/raisin-binary-classification/versions/1
File saved to ./drive/MyDrive/datasets/raisan.csv


In [20]:
data_path = "./drive/MyDrive/datasets/raisin.csv"
df = pd.read_csv(data_path)

df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [22]:
df_X = df.drop(columns=["Class"])
df_y = df["Class"]

if df_y.dtype == "object":
  df_y = df_y.map({"Kecimen": 0, "Besni": 1})
  df_y.astype(int)

df_y

,Class
0,0
1,0
2,0
3,0
4,0
...,...
895,1
896,1
897,1
898,1


In [23]:
# manually scale

X_values = df_X.values
mu = X_values.mean(axis=0)
sigma = X_values.std(axis=0)

X_values = (X_values - mu) / sigma

df_X_m_scaled = pd.DataFrame(X_values, columns=df_X.columns)
df_X_m_scaled.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,-0.007186,0.097577,-0.023958,0.423142,-0.015709,1.106743,0.066274
1,-0.324217,-0.209012,-0.229292,0.224476,-0.304248,-0.287777,-0.161252
2,0.078292,0.097758,0.236988,0.186239,0.062113,-1.158250,0.155945
3,-1.074286,-1.245051,-0.915273,-1.069623,-1.076165,0.001711,-1.175915
4,-0.215393,-0.678958,0.727354,-2.409827,-0.238623,1.745259,-0.338639


In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_X)
df_X_scaled = pd.DataFrame(X_scaled, columns=df_X.columns)
df_X_scaled.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,-0.007186,0.097577,-0.023958,0.423142,-0.015709,1.106743,0.066274
1,-0.324217,-0.209012,-0.229292,0.224476,-0.304248,-0.287777,-0.161252
2,0.078292,0.097758,0.236988,0.186239,0.062113,-1.158250,0.155945
3,-1.074286,-1.245051,-0.915273,-1.069623,-1.076165,0.001711,-1.175915
4,-0.215393,-0.678958,0.727354,-2.409827,-0.238623,1.745259,-0.338639


In [28]:
### Properly scaled with split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


y_train, y_test = y_train.to_numpy(), y_test.to_numpy()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((720, 7), (180, 7), (720,), (180,))

# Gradient Boosting Machines (GBM) — Mathematical Overview

Gradient Boosting is an ensemble method that builds a strong predictive model by sequentially combining weak learners, typically decision trees. Each new tree corrects the errors (residuals) of the combined ensemble so far.

---

## 1️⃣ Model Representation

For a dataset $(x_i, y_i), i=1,...,N$, GBM models the target as a sum of weak learners:

$$
F_M(x) = \sum_{m=1}^M \eta \, h_m(x)
$$

Where:  

- $M$ = number of trees (boosting rounds)  
- $\eta \in (0,1]$ = learning rate  
- $h_m(x)$ = m-th weak learner (regression tree)  
- $F_0(x)$ = initial prediction (often mean of target for regression or log-odds for classification)

---

## 2️⃣ Loss Function

GBM minimizes a differentiable loss function $L(y, F(x))$.  

- Regression (squared error loss):  

$$
L(y, F(x)) = \frac{1}{2} (y - F(x))^2
$$

- Binary classification (logistic loss):  

$$
L(y, F(x)) = - \big[y \log p + (1-y) \log (1-p)\big], \quad p = \sigma(F(x)) = \frac{1}{1+e^{-F(x)}}
$$

---

## 3️⃣ Compute Pseudo-Residuals (Gradients)

At iteration $m$, compute the negative gradient of the loss w.r.t. current predictions:

$$
r_{im} = - \left. \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right|_{F = F_{m-1}(x_i)}
$$

- Regression (squared error):  

$$
r_{im} = y_i - F_{m-1}(x_i)
$$

- Binary logistic loss:  

$$
r_{im} = y_i - \sigma(F_{m-1}(x_i))
$$

These pseudo-residuals become the target for the new tree.

---

## 4️⃣ Fit a Regression Tree

Train a regression tree $h_m(x)$ to predict the pseudo-residuals $r_{im}$ from features $x_i$.  

- Each leaf $j$ has a prediction value $\gamma_{jm}$.  
- For squared error loss:  

$$
\gamma_{jm} = \text{mean of } r_{im} \text{ in leaf } j
$$

- For general loss (e.g., logistic), leaf value is found via:  

$$
\gamma_{jm} = \arg\min_\gamma \sum_{i \in R_{jm}} L\big(y_i, F_{m-1}(x_i) + \gamma\big)
$$

where $R_{jm}$ = set of samples in leaf $j$.

---

## 5️⃣ Update Model Predictions

After fitting the tree, update the ensemble:

$$
F_m(x) = F_{m-1}(x) + \eta \, h_m(x)
$$

- $\eta$ scales the contribution of each tree (prevents overfitting).  
- For classification, probabilities are computed from logits:

$$
p(x) = \sigma(F_M(x)) = \frac{1}{1 + e^{-F_M(x)}}
$$

---

## 6️⃣ Splitting Criterion (Tree Construction)

- In each node, find the split that maximizes reduction in loss.  
- For regression with squared error:  

$$
\text{SSE}_\text{node} = \sum_{i \in \text{node}} (r_{im} - \bar{r}_{\text{node}})^2
$$

- For general differentiable loss (XGBoost-style):  

$$
\text{Gain} = \frac{1}{2} \left( \frac{(\sum g_L)^2}{\sum h_L + \lambda} + \frac{(\sum g_R)^2}{\sum h_R + \lambda} - \frac{(\sum g)^2}{\sum h + \lambda} \right)
$$

Where:  

- $g$ = gradient, $h$ = Hessian (second derivative)  
- $L, R$ = left/right child nodes  

---

## 7️⃣ Summary of Steps per Boosting Round

1. Compute pseudo-residuals: $r_{im} = - \frac{\partial L}{\partial F_{m-1}(x_i)}$  
2. Fit regression tree to $r_{im}$  
3. Compute leaf values $\gamma_{jm}$ by minimizing loss in each leaf  
4. Update predictions: $F_m(x) = F_{m-1}(x) + \eta h_m(x)$  
5. Repeat for $m=1,...,M$

Final prediction: $F_M(x)$, optionally passed through sigmoid for classification.


In [4]:
class Node:

  def __init__(self, feature= None, threshold=None, value=None):
    self.feature = feature
    self.threshold = threshold
    self.value = value
    self.left = None
    self.right = None


In [5]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

def compute_residuals(y,f):
  p = sigmoid(f)
  return y - p

In [34]:
def best_split(X,residuals):
  m,n = X.shape
  best_feature = None
  best_threshold = None
  best_gain = float("inf")

  for feature in range(n):
    thresholds = np.unique(X[:,feature])
    for thresh in thresholds:
      left_idx = X[:, feature] <= thresh
      right_idx = X[:, feature] > thresh

      if left_idx.sum() == 0 or right_idx.sum() == 0:
        continue

      # MSE
      left_loss = np.var(residuals[left_idx]) * left_idx.sum()
      right_loss = np.var(residuals[right_idx]) * right_idx.sum()
      loss = left_loss + right_loss

      if loss < best_gain:
        best_feature = feature
        best_threshold = thresh
        best_gain = loss

  return best_feature, best_threshold

In [7]:
def build_tree(X,residuals,depth=0,max_depth=10):
  m,n = X.shape

  if depth>=max_depth or np.var(residuals)<1e-6:
    return Node(value=np.mean(residuals))

  feature, threshold = best_split(X,residuals)

  if feature is None:
    return Node(value=np.mean(residuals))

  node =Node(feature=feature,threshold=threshold)

  left_idx = X[:,feature] <= threshold
  right_idx = ~left_idx

  node.left = build_tree(X[left_idx],residuals[left_idx],depth+1,max_depth)
  node.right = build_tree(X[right_idx],residuals[right_idx],depth+1,max_depth)

  return node


In [9]:
def predict_tree(node,x):
  if node.value is not None:
    return node.value
  if x[node.feature] <= node.threshold:
    return predict_tree(node.left,x)
  else:
    return predict_tree(node.right,x)

def predict_all(node,X):
  return np.array([predict_tree(node,x) for x in X])

In [40]:
def train_gbm(X_train,y_train, n_estimators=10, lr=0.1, max_depth = 2, verbose=True):
  f_train = np.zeros(len(y_train))
  trees = []

  for i in range(n_estimators):
    residuals = compute_residuals(y_train,f_train)

    tree = build_tree(X_train,residuals,max_depth=max_depth)
    trees.append(tree)

    pred_residuals =  predict_all(tree,X_train)
    f_train += lr * pred_residuals

    if verbose and ((i+1)%5 == 0 or i ==0):
      p_train = sigmoid(f_train)
      logloss = -np.mean(y_train*np.log(p_train+1e-9) + (1-y_train)*np.log(1-p_train+1e-9))
      print(f"Round {i+1}/{n_estimators}, Log-loss: {logloss:.4f}")


  return trees, f_train


In [30]:
def predict_gbm(X, trees, lr=0.1):
  f = np.zeros(X.shape[0])
  for tree in trees:
    pred = predict_all(tree,X)
    f += lr * pred

  return (sigmoid(f)>0.5).astype(int)

In [43]:
trees, f_train = train_gbm(X_train, y_train, n_estimators=100, lr=0.1, max_depth=2)

# Predict on test set
y_pred = predict_gbm(X_test, trees, lr=0.1)

# Evaluate
from sklearn.metrics import accuracy_score
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Round 1/100, Log-loss: 0.6783
Round 5/100, Log-loss: 0.6260
Round 10/100, Log-loss: 0.5737
Round 15/100, Log-loss: 0.5326
Round 20/100, Log-loss: 0.4998
Round 25/100, Log-loss: 0.4719
Round 30/100, Log-loss: 0.4498
Round 35/100, Log-loss: 0.4311
Round 40/100, Log-loss: 0.4159
Round 45/100, Log-loss: 0.4031
Round 50/100, Log-loss: 0.3923
Round 55/100, Log-loss: 0.3829
Round 60/100, Log-loss: 0.3745
Round 65/100, Log-loss: 0.3672
Round 70/100, Log-loss: 0.3607
Round 75/100, Log-loss: 0.3550
Round 80/100, Log-loss: 0.3497
Round 85/100, Log-loss: 0.3449
Round 90/100, Log-loss: 0.3405
Round 95/100, Log-loss: 0.3364
Round 100/100, Log-loss: 0.3326
Test Accuracy: 0.8611111111111112


In [44]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=100,
                                 learning_rate=0.1,
                                 max_depth=2,
                                 random_state=42)

gbm.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=2, random_state=42)

In [45]:
# Predicted probabilities
y_prob = gbm.predict_proba(X_test)[:, 1]

# Predicted class labels
y_pred = gbm.predict(X_test)
from sklearn.metrics import accuracy_score, log_loss

acc = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, y_prob)

print("Test Accuracy:", acc)
print("Test Log-loss:", loss)

Test Accuracy: 0.8666666666666667
Test Log-loss: 0.32931104578163645
